In [3]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [9]:
# os = os.getcwd()
# print(os)

#not sure why path to read was Project_Two instead of Project_Two/ML Model Notebooks

crypto_df = pd.read_csv(
    Path("Resources/EthData.csv"),
    index_col='Date', 
    infer_datetime_format=True, 
    parse_dates=True
)



crypto_df.head()



,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2022-10-30,1619.152889,1637.016000,1578.251269,1589.301300,2.689797e+10,1.964897e+11
2022-10-29,1554.409800,1644.768781,1549.777300,1620.511904,3.354897e+10,1.970151e+11
2022-10-28,1514.216064,1568.541700,1490.825800,1556.203000,3.610905e+10,1.871443e+11
2022-10-27,1566.498187,1573.201509,1510.528142,1514.690285,4.145711e+10,1.900150e+11
2022-10-26,1459.724557,1583.129032,1459.028925,1565.494644,5.386308e+10,1.872332e+11


In [10]:
#crypto_df = crypto_df.sort_values(by=['Date'], ascending = True)

In [11]:
crypto_df.fillna(0, inplace=True)

lags = 5

for i in range(0, lags):
    crypto_df['Lag_' + str(i+1)] = crypto_df['Close'].shift(i+1)
    
crypto_df['Returns'] = crypto_df['Close'].pct_change()
    
crypto_df.head(10)

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns
Date,,,,,,,,,,,,
2022-10-30,1619.152889,1637.016000,1578.251269,1589.301300,2.689797e+10,1.964897e+11,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-29,1554.409800,1644.768781,1549.777300,1620.511904,3.354897e+10,1.970151e+11,1589.301300,NaN,NaN,NaN,NaN,0.019638
2022-10-28,1514.216064,1568.541700,1490.825800,1556.203000,3.610905e+10,1.871443e+11,1620.511904,1589.301300,NaN,NaN,NaN,-0.039684
2022-10-27,1566.498187,1573.201509,1510.528142,1514.690285,4.145711e+10,1.900150e+11,1556.203000,1620.511904,1589.301300,NaN,NaN,-0.026676
2022-10-26,1459.724557,1583.129032,1459.028925,1565.494644,5.386308e+10,1.872332e+11,1514.690285,1556.203000,1620.511904,1589.301300,NaN,0.033541
2022-10-25,1343.310600,1510.181700,1287.079560,1463.420466,2.612383e+10,1.702375e+11,1565.494644,1514.690285,1556.203000,1620.511904,1589.301300,-0.065203
2022-10-24,1363.093200,1367.684300,1326.899600,1343.697200,2.117332e+10,1.648366e+11,1463.420466,1565.494644,1514.690285,1556.203000,1620.511904,-0.081811
2022-10-23,1313.018600,1368.126600,1302.270300,1361.791800,1.109858e+10,1.615219e+11,1343.697200,1463.420466,1565.494644,1514.690285,1556.203000,0.013466
2022-10-22,1299.598000,1317.646618,1295.022000,1313.018181,1.530541e+10,1.600873e+11,1361.791800,1343.697200,1463.420466,1565.494644,1514.690285,-0.035816


In [12]:
crypto_df['Direction'] = np.sign(crypto_df['Returns'])

crypto_df = crypto_df.dropna()
crypto_df = crypto_df.drop(crypto_df.index[0:5])

##remove rows with any values that are not infinite
crypto_df = crypto_df[np.isfinite(crypto_df).all(1)]

crypto_df

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns,Direction
Date,,,,,,,,,,,,,
2022-10-20,1285.696900,1308.281500,1274.465400,1282.688400,1.749868e+10,1.580914e+11,1299.529200,1313.018181,1361.791800,1343.697200,1463.420466,-0.012959,-1.0
2022-10-19,1310.305500,1311.852600,1283.771500,1284.617000,1.972740e+10,1.592551e+11,1282.688400,1299.529200,1313.018181,1361.791800,1343.697200,0.001504,1.0
2022-10-18,1330.853900,1339.502400,1291.118500,1311.931500,2.047246e+10,1.619986e+11,1284.617000,1282.688400,1299.529200,1313.018181,1361.791800,0.021263,1.0
2022-10-17,1305.280202,1337.124100,1296.595120,1332.405800,1.782394e+10,1.614012e+11,1311.931500,1284.617000,1282.688400,1299.529200,1313.018181,0.015606,1.0
2022-10-16,1275.005500,1313.207146,1275.005500,1305.740349,1.244793e+10,1.578903e+11,1332.405800,1311.931500,1284.617000,1282.688400,1299.529200,-0.020013,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-08-12,1.067860,1.289940,0.883608,1.237670,1.971841e+06,6.682610e+07,1.828190,1.832310,1.692440,1.512680,1.200730,-0.323008,-1.0
2015-08-11,0.708448,1.131410,0.663235,1.057670,6.406322e+05,4.768204e+07,1.237670,1.828190,1.832310,1.692440,1.512680,-0.145435,-1.0
2015-08-10,0.701897,0.729854,0.636546,0.707703,4.577479e+05,4.182758e+07,1.057670,1.237670,1.828190,1.832310,1.692440,-0.330885,-1.0


In [13]:
X = crypto_df[['Lag_1', 'Lag_2']]
y = crypto_df['Direction']

crypto_df.head(10)

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns,Direction
Date,,,,,,,,,,,,,
2022-10-20,1285.696900,1308.281500,1274.465400,1282.688400,1.749868e+10,1.580914e+11,1299.529200,1313.018181,1361.791800,1343.697200,1463.420466,-0.012959,-1.0
2022-10-19,1310.305500,1311.852600,1283.771500,1284.617000,1.972740e+10,1.592551e+11,1282.688400,1299.529200,1313.018181,1361.791800,1343.697200,0.001504,1.0
2022-10-18,1330.853900,1339.502400,1291.118500,1311.931500,2.047246e+10,1.619986e+11,1284.617000,1282.688400,1299.529200,1313.018181,1361.791800,0.021263,1.0
2022-10-17,1305.280202,1337.124100,1296.595120,1332.405800,1.782394e+10,1.614012e+11,1311.931500,1284.617000,1282.688400,1299.529200,1313.018181,0.015606,1.0
2022-10-16,1275.005500,1313.207146,1275.005500,1305.740349,1.244793e+10,1.578903e+11,1332.405800,1311.931500,1284.617000,1282.688400,1299.529200,-0.020013,-1.0
2022-10-15,1296.634000,1300.896028,1269.229700,1273.866000,1.820297e+10,1.578972e+11,1305.740349,1332.405800,1311.931500,1284.617000,1282.688400,-0.024411,-1.0
2022-10-14,1287.291287,1338.382300,1284.910682,1297.552300,3.345885e+10,1.612200e+11,1273.866000,1305.740349,1332.405800,1311.931500,1284.617000,0.018594,1.0
2022-10-13,1294.896552,1297.824797,1208.511000,1288.082946,2.384634e+10,1.559701e+11,1297.552300,1273.866000,1305.740349,1332.405800,1311.931500,-0.007298,-1.0
2022-10-12,1280.261800,1303.791700,1277.677700,1294.894777,1.709829e+10,1.586980e+11,1288.082946,1297.552300,1273.866000,1305.740349,1332.405800,0.005288,1.0


In [14]:
start_test = pd.Timestamp("2015-08-10")

X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]



In [15]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [16]:
lr_accuracy = accuracy_score(y_test, y_pred)
lr_accuracy

0.48611639406618484

In [17]:
pred = (1.0 + (y_pred == y_test))/2.0
hit_rate = np.mean(pred)
print('Logistic Regression - Hit Rate of Ethereum: {:.4f}'.format(hit_rate))

Logistic Regression - Hit Rate of Ethereum: 0.7431
